## Loading a Fine-Tuned Detectron Model for Auto Labeling
In this notebook, we will load a finely tuned Detectron model and use it to label datasets and upload them back to SegmentsAI.



In [2]:
import numpy as np

# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()


# checking the version and if we have cuda available
from src.segmentation.framework_handlers.detectron2_handler import print_version_info

!nvcc --version
print_version_info()

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_Sep_13_19:13:29_PDT_2021
Cuda compilation tools, release 11.5, V11.5.50
Build cuda_11.5.r11.5/compiler.30411180_0
torch:  1.9 ; cuda:  11.1
detectron2: 0.6


In [4]:
# define the time of the training
from datetime import datetime

# Get the current date and time of the model training process
current_time = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")

print(f"current time {current_time}")

current time 06-04-2024_15-08-40


Code for loading model checkpoint and config

In [5]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

# load the best model config and weights
checkpoint_path = "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth"
config_yaml_path = "/home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/config.yaml"
cfg = get_cfg()
# load config
cfg.merge_from_file(config_yaml_path)
# define predictor
# load checkpoint
cfg.MODEL.WEIGHTS = checkpoint_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

Loading config /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/config.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[04/06 15:09:35 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /home/etaylor/code_projects/thesis/checkpoints/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/30-12-2023_12-34-05_all_train_images/model_final.pth ...


In [9]:
# load segmentsai handlers
from src.annotation_handling.segmentsai_handler import SegmentsAIHandler
from src.pipelines import model_assist_label_pipeline as pipe 
import config
import time
from segments import SegmentsDataset

segmentsai_handler = SegmentsAIHandler()


add a new dataset to segments and predict annotations using the model trained

In [12]:
from src.annotation_handling import annotation_handler

# New dataset to upload
image_name = "IMG_0041"
week = 'week5'
zoom_type = '3xr'
release = "v0.1"
nir_username = "Nirmalka"
dorin_username = "dorins"
collaborator = nir_username

# Create a new test dataset for the specified image, week, and zoom type
test_dataset = pipe.create_new_test_dataset(image_name, config.WEEKS_DIR[week], config.ZOOM_TYPES_DIR[zoom_type], single_category=False)
# # Get the absolute path to the processed image
abs_images_path = f"{config.get_processed_cannabis_image_path(week, zoom_type)}/{image_name}"
raw_images_path = f"{config.get_raw_image_path(week, zoom_type)}/{image_name}.JPG"

# Upload the images that are not annotated to the dataset
segmentsai_handler.upload_single_image(test_dataset, raw_images_path) # upload the raw image path
segmentsai_handler.upload_images(test_dataset, abs_images_path)

description = "upload predictions to dataset."
segmentsai_handler.client.add_release(test_dataset, release, description)
print("Waiting 5 seconds that the release is created...")
time.sleep(5)
test_release = segmentsai_handler.client.get_release(test_dataset, "v0.1")
print(test_release)

# Add collaborator to the dataset
segmentsai_handler.client.add_dataset_collaborator(
    dataset_identifier=test_dataset,
    username=collaborator
)

test_dataset_instance = SegmentsDataset(test_release)

for sample in test_dataset_instance:
    # Generate label predictions
    image = np.array(sample["image"])
    # Change RGB (PIL) to BGR (OpenCV)
    cv2_image = image[:, :, ::-1]
    outputs = predictor(cv2_image)
    
    segmentation_bitmap, annotations = annotation_handler.convert_coco_to_segments_format(cv2_image, outputs)
    segmentsai_handler.upload_annotation_for_sample(sample['uuid'], segmentation_bitmap, annotations)

name='cannabis_patches_week5_18_05_2023_3x_regular_IMG_0041' full_name='etaylor/cannabis_patches_week5_18_05_2023_3x_regular_IMG_0041' cloned_from=None description='cannabis patches week=week5_18_05_2023 zoom_type=3x_regular of image=IMG_0041.' category='other' public=False owner=Owner(username='etaylor', created_at='2022-12-28T12:53:18Z', email=None) created_at='2024-04-06T12:27:56.426421Z' enable_ratings=False enable_skip_labeling=True enable_skip_reviewing=False enable_save_button=False enable_label_status_verified=False enable_same_dimensions_track_constraint=False enable_interpolation=True task_type='segmentation-bitmap' label_stats=LabelStats(REVIEWED=None, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=None, PRELABELED=None, SKIPPED=None, VERIFIED=None, UNLABELED=None, TOTAL=None) labeling_inactivity_timeout_seconds=300 samples_count=0 collaborators_count=None task_attributes=TaskAttributes(format_version='0.1', categories=[TaskAttributeCategory(na

100%|██████████| 10/10 [00:04<00:00,  2.43it/s]


Initialized dataset with 10 images.
